<a href="https://colab.research.google.com/github/engcarlo/TransferLearning-Datasets/blob/main/M2_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning / fine-tuning

This tutorial will guide you through the process of using _transfer learning_ to learn an accurate image classifier from a relatively small number of training samples. Generally speaking, transfer learning refers to the process of leveraging the knowledge learned in one model for the training of another model.

More specifically, the process involves taking an existing neural network which was previously trained to good performance on a larger dataset, and using it as the basis for a new model which leverages that previous network's accuracy for a new task. This method has become popular in recent years to improve the performance of a neural net trained on a small dataset; the intuition is that the new dataset may be too small to train to good performance by itself, but we know that most neural nets trained to learn image features often learn similar features anyway, especially at early layers where they are more generic (edge detectors, blobs, and so on).

Transfer learning has been largely enabled by the open-sourcing of state-of-the-art models; for the top performing models in image classification tasks (like from [ILSVRC](http://www.image-net.org/challenges/LSVRC/)), it is common practice now to not only publish the architecture, but to release the trained weights of the model as well. This lets amateurs use these top image classifiers to boost the performance of their own task-specific models.

#### Feature extraction vs. fine-tuning

At one extreme, transfer learning can involve taking the pre-trained network and freezing the weights, and using one of its hidden layers (usually the last one) as a feature extractor, using those features as the input to a smaller neural net.

At the other extreme, we start with the pre-trained network, but we allow some of the weights (usually the last layer or last few layers) to be modified. Another name for this procedure is called "fine-tuning" because we are slightly adjusting the pre-trained net's weights to the new task. We usually train such a network with a lower learning rate, since we expect the features are already relatively good and do not need to be changed too much.

Sometimes, we do something in-between: Freeze just the early/generic layers, but fine-tune the later layers. Which strategy is best depends on the size of your dataset, the number of classes, and how much it resembles the dataset the previous model was trained on (and thus, whether it can benefit from the same learned feature extractors). A more detailed discussion of how to strategize can be found in [[1]](http://cs231n.github.io/transfer-learning/) [[2]](http://sebastianruder.com/transfer-learning/).

## Procedure

In this guide will go through the process of loading a state-of-the-art, 1000-class image classifier, [VGG16](https://arxiv.org/pdf/1409.1556.pdf) which [won the ImageNet challenge in 2014](http://www.robots.ox.ac.uk/~vgg/research/very_deep/), and using it as a fixed feature extractor to train a smaller custom classifier on our own images, although with very few code changes, you can try fine-tuning as well.

We will first load VGG16 and remove its final layer, the 1000-class softmax classification layer specific to ImageNet, and replace it with a new classification layer for the classes we are training over. We will then freeze all the weights in the network except the new ones connecting to the new classification layer, and then train the new classification layer over our new dataset.

We will also compare this method to training a small neural network from scratch on the new dataset, and as we shall see, it will dramatically improve our accuracy. We will do that part first.

As our test subject, we'll use a dataset consisting of around 6000 images belonging to 97 classes, and train an image classifier with around 80% accuracy on it. It's worth noting that this strategy scales well to image sets where you may have even just a couple hundred or less images. Its performance will be lesser from a small number of samples (depending on classes) as usual, but still impressive considering the usual constraints.


### Load Libraries

In [ ]:
%matplotlib inline

import os

#if using Theano with GPU
#os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input
from keras.models import Model

### Libraries List and Requirements File

In [ ]:
pip list

In [ ]:
pip freeze > requirements.txt

### Getting a dataset

The first step is going to be to load our data. As our example, we will be using the dataset **Cats vs. Dogs**. This dataset is a classic image classification dataset containing images of cats and dogs, and is suitable for demonstrating transfer learning techniques.

To obtain this dataset, you can download it from the Github Repository using the following link:

[https://github.com/engcarlo/TransferLearning-Datasets/tree/main/Dataset](https://github.com/engcarlo/TransferLearning-Datasets/tree/main/Dataset)

Alternatively, it can be found in may Kaggle profile dataset using the following link:

[https://www.kaggle.com/datasets/quantyukio/cats-and-dogs-sample2transfer-learning](https://www.kaggle.com/datasets/quantyukio/cats-and-dogs-sample2transfer-learning)

Moreover, if you have already downloaded and organized the dataset, ensure it is arranged in a similar fashion to the structure expected by the code in this notebook, with images organized into subfolders, one for each class (e.g., 'Cats' and 'Dogs'). If you wish to use your own dataset with a different structure, you will need to modify the data loading code to correctly load your images and their corresponding labels. The helper function `get_image(path)` can be used to load and preprocess individual images to the required 224x224 size for the VGG16 model.

If your dataset is organized with subfolders for each class, the following cell should load all the data correctly by just replacing `root` with the path to your dataset folder.

In [ ]:
user = "engcarlo"
repo = "TransferLearning-Datasets"

# remove local directory if it already exists
if os.path.isdir(repo):
    !rm -rf {repo}

!git clone https://github.com/{user}/{repo}.git

### Dataset Description

The dataset loaded from the `TransferLearning-Datasets` repository contains images belonging to two categories: 'Cats' and 'Dogs'. This dataset is used to train and evaluate image classification models, specifically for demonstrating transfer learning techniques.

The data is organized into subfolders within the `/content/TransferLearning-Datasets/Dataset` directory, with each subfolder representing a class ('Cats' and 'Dogs'). The images are loaded and pre-processed to a target size of 224x224 pixels, which is the required input size for the VGG16 model used in this tutorial.

The dataset is split into training, validation, and test sets to facilitate model training and evaluation.

In [ ]:
set_train_split = 0.8 # @param {type: "slider", min: 0.0, max: 1.0, step: 0.01}
set_val_split = (1-set_train_split)/2

root = '/content/TransferLearning-Datasets/Dataset'
train_split, val_split = set_train_split, set_val_split
categories = [x[0] for x in os.walk(root) if x[0]][1:]

print(f"""
Train Set:      {round(set_train_split*100, 2)} %
Validation Set: {round(set_val_split*100, 2)} %
Test Set:       {round(set_val_split*100, 2)} %
Directory:
{categories}
""")

This function is useful for pre-processing the data into an image and input vector.

In [ ]:
# helper function to load image and return it and input vector
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

Load all the images from root folder

In [ ]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames
              in os.walk(category) for f in filenames
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

# count the number of classes
num_classes = len(categories)

Randomize the data order.

In [ ]:
random.shuffle(data)

create training / validation / test split (80%, 10%, 10%)

In [ ]:
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]

Separate data for labels.

In [ ]:
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]
print(y_test)

Pre-process the data as before by making sure it's float32 and normalized between 0 and 1.

In [ ]:
# normalize data
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# convert labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape)

Let's get a summary of what we have.

In [ ]:
# summary
print("finished loading %d images from %d categories"%(len(data), num_classes))
print("train / validation / test split: %d, %d, %d"%(len(x_train), len(x_val), len(x_test)))
print("training data shape: ", x_train.shape)
print("training labels shape: ", y_train.shape)


If everything worked properly, you should have loaded a bunch of images, and split them into three sets: `train`, `val`, and `test`. The shape of the training data should be (`n`, 224, 224, 3) where `n` is the size of your training set, and the labels should be (`n`, `c`) where `c` is the number of classes (2 in the case of `TransferLearning-Datasets`.

Notice that we divided all the data into three subsets -- a training set `train`, a validation set `val`, and a test set `test`. The reason for this is to properly evaluate the accuracy of our classifier. During training, the optimizer uses the validation set to evaluate its internal performance, in order to determine the gradient without overfitting to the training set. The `test` set is always held out from the training algorithm, and is only used at the end to evaluate the final accuracy of our model.

Let's quickly look at a few sample images from our dataset.

In [ ]:
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(root) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
idx = [int(len(images) * random.random()) for i in range(8)]
imgs = [image.load_img(images[i], target_size=(224, 224)) for i in idx]
concat_image = np.concatenate([np.asarray(img) for img in imgs], axis=1)
plt.figure(figsize=(25,5))
plt.imshow(concat_image)

## First training a neural net from scratch

Before doing the transfer learning, let's first build a neural network from scratch for doing classification on our dataset. This will give us a baseline to compare to our transfer-learned network later.

The network we will construct contains 4 alternating convolutional and max-pooling layers, followed by a [dropout](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) after every other conv/pooling pair. After the last pooling layer, we will attach a fully-connected layer with 256 neurons, another dropout layer, then finally a softmax classification layer for our classes.

Our loss function will be, as usual, categorical cross-entropy loss, and our learning algorithm will be [AdaDelta](https://arxiv.org/abs/1212.5701). Various things about this network can be changed to get better performance, perhaps using a larger network or a different optimizer will help, but for the purposes of this notebook, the goal is to just get an understanding of an approximate baseline for comparison's sake, and so it isn't neccessary to spend much time trying to optimize this network.

Upon compiling the network, let's run `model.summary()` to get a snapshot of its layers.

# Building the Neural Network

This section defines the architecture of the neural network that will be trained from scratch. The network is built using the `Sequential` model from Keras, which allows for a linear stack of layers.

The network consists of the following layers:

- **Convolutional Layers (`Conv2D`)**: These layers apply convolution filters to the input image to extract features. The number of filters in each convolutional layer is set using the `set_conv2D_x` variables defined in the previous cell. The kernel size is set to (3, 3).
- **Activation Layers (`Activation('relu')`)**: A ReLU (Rectified Linear Unit) activation function is applied after each convolutional layer to introduce non-linearity.
- **MaxPooling Layers (`MaxPooling2D`)**: These layers downsample the feature maps, reducing their spatial dimensions and the number of parameters. The pool size is set to (2, 2).
- **Dropout Layers (`Dropout`)**: These layers randomly set a fraction of the input units to zero during training, which helps prevent overfitting.
- **Flatten Layer (`Flatten`)**: This layer reshapes the output from the convolutional and pooling layers into a 1D vector to be fed into the dense layers.
- **Dense Layers (`Dense`)**: These are fully connected layers. The first dense layer has 256 neurons, and the final dense layer has `num_classes` neurons, corresponding to the number of categories in the dataset.
- **Activation Layer (`Activation('softmax')`)**: A softmax activation function is applied to the output of the final dense layer to produce probability distributions over the classes.

The `input_shape` for the first convolutional layer is set to `x_train.shape[1:]`, which corresponds to the dimensions of the input images (height, width, channels). The parameters for the convolutional and dense layers (weights and biases) are initialized randomly by Keras when the model is compiled.

#### Set Neural Network Parameters

In [ ]:
set_kernel_dim = 3 # @param {type: 'integer'}
C2D_Kernel_size = (set_kernel_dim, set_kernel_dim)
set_conv2D_0 = "32" # @param  [8, 16, 32, 64, 128, 256, 512]
set_conv2D_0 = int(set_conv2D_0)
set_conv2D_1 = "32" # @param [8, 16, 32, 64, 128, 256, 512]
set_conv2D_1 = int(set_conv2D_1)
set_conv2D_2 = "32" # @param [8, 16, 32, 64, 128, 256, 512]
set_conv2D_2 = int(set_conv2D_2)
set_conv2D_3 = "32" # @param [8, 16, 32, 64, 128, 256, 512]
set_conv2D_3 = int(set_conv2D_3)

set_max_pool_dim_0 = 2 # @param {type: 'integer'}
MP_dimXY_0 = (set_max_pool_dim_0, set_max_pool_dim_0)
set_max_pool_dim_1 = 2 # @param {type: 'integer'}
MP_dimXY_1 = (set_max_pool_dim_1, set_max_pool_dim_1)
set_max_pool_dim_2 = 2 # @param {type: 'integer'}
MP_dimXY_2 = (set_max_pool_dim_2, set_max_pool_dim_2)
set_max_pool_dim_3 = 2 # @param {type: 'integer'}
MP_dimXY_3 = (set_max_pool_dim_3, set_max_pool_dim_3)

# Dropout Fraction
set_dropout_0 = 0.2 # @param {type: 'slider', min: 0.0, max: 1.0, step: 0.05}
set_dropout_1 = 0.2 # @param {type: 'slider', min: 0.0, max: 1.0, step: 0.05}
set_dropout_2 = 0.4 # @param {type: 'slider', min: 0.0, max: 1.0, step: 0.05}

# Activation Method
set_activation_0 = 'tanh' # @param ["relu", "sigmoid", "tanh"]
set_activation_1 = 'tanh' # @param ["relu", "sigmoid", "tanh"]
set_activation_2 = 'tanh' # @param ["relu", "sigmoid", "tanh"]
set_activation_3 = 'tanh' # @param ["relu", "sigmoid", "tanh"]
set_activation_4 = 'tanh' # @param ["relu", "sigmoid", "tanh"]
set_activation_5 = 'softmax' # @param ["softmax", "relu", "sigmoid", "tanh"]

# Dense Neurons
set_dense_neurons_0 = 512 # @param {type: 'integer'}

In [ ]:
# build the network
model = Sequential()
print("Input dimensions: ", x_train.shape[1:])

# Add the Input layer explicitly
model.add(Input(shape=x_train.shape[1:]))

model.add(Conv2D(set_conv2D_0, C2D_Kernel_size))
#model.add(Conv2D(set_conv2D_0, C2D_Kernel_size))
model.add(Activation(set_activation_0))
model.add(MaxPooling2D(pool_size = MP_dimXY_0))

model.add(Conv2D(set_conv2D_1, C2D_Kernel_size))
#model.add(Conv2D(set_conv2D_1, C2D_Kernel_size))
model.add(Activation(set_activation_1))
model.add(MaxPooling2D(pool_size = MP_dimXY_1))

model.add(Dropout(set_dropout_0))

model.add(Conv2D(set_conv2D_2, C2D_Kernel_size))
#model.add(Conv2D(set_conv2D_2, C2D_Kernel_size))
model.add(Activation(set_activation_2))
model.add(MaxPooling2D(pool_size = MP_dimXY_2))

model.add(Conv2D(set_conv2D_3, C2D_Kernel_size))
#model.add(Conv2D(set_conv2D_3, C2D_Kernel_size))
model.add(Activation(set_activation_3))
model.add(MaxPooling2D(pool_size = MP_dimXY_3))

model.add(Dropout(set_dropout_1))

model.add(Flatten())
model.add(Dense(set_dense_neurons_0))
model.add(Activation(set_activation_4))

model.add(Dropout(set_dropout_2))

model.add(Dense(num_classes))
model.add(Activation(set_activation_5))

model.summary()

We've created a medium-sized network with ~1.2 million weights and biases (the parameters). Most of them are leading into the one pre-softmax fully-connected layer "dense_5".

We can now go ahead and train our model for 100 epochs with a batch size of 128. We'll also record its history so we can plot the loss over time later.

In [ ]:
# Set Parameters
# Default: batch_size = 128, epochs = 10, optmizer = "adam", loss = "categorical_crossentropy"
set_batch_size  = 128  # @param {type:"integer"}
set_epochs      = 50       # @param {type:"integer"}
set_optmizer    = 'sgd' # @param ["adam", "rmsprop", "sgd"]
set_loss        = 'categorical_crossentropy' # @param ["categorical_crossentropy", "binary_crossentropy", "mse"]

#
# compile the model to use categorical cross-entropy loss function and adadelta optimizer
model.compile(loss        = set_loss,
              optimizer   = set_optmizer,
              metrics     = ['accuracy'])
history = model.fit(x_train,
                    y_train,
                    batch_size = set_batch_size,
                    epochs     = set_epochs,
                    validation_data = (x_val, y_val))


Let's plot the validation loss and validation accuracy over time.

In [ ]:
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.set_title("validation loss")
ax.set_xlabel("epochs")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_accuracy"])
ax2.set_title("validation accuracy")
ax2.set_xlabel("epochs")
ax2.set_ylim(0, 1)

plt.show()

Notice that the validation loss begins to actually rise after around 16 epochs, even though validation accuracy remains roughly between 40% and 50%. This suggests our model begins overfitting around then, and best performance would have been achieved if we had stopped early around then. Nevertheless, our accuracy would not have likely been above 50%, and probably lower down.

We can also get a final evaluation by running our model on the training set. Doing so, we get the following results:

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Finally, we see that we have achieved a (top-1) accuracy of around 49%. That's not too bad for 6000 images, considering that if we were to use a naive strategy of taking random guesses, we would have only gotten around 1% accuracy.

# Transfer learning by starting with existing network

Now we can move on to the main strategy for training an image classifier on our small dataset: by starting with a larger and already trained network.

To start, we will load the VGG16 from keras, which was trained on ImageNet and the weights saved online. If this is your first time loading VGG16, you'll need to wait a bit for the weights to download from the web. Once the network is loaded, we can again inspect the layers with the `summary()` method.

In [ ]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

Notice that VGG16 is _much_ bigger than the network we constructed earlier. It contains 13 convolutional layers and two fully connected layers at the end, and has over 138 million parameters, around 100 times as many parameters than the network we made above. Like our first network, the majority of the parameters are stored in the connections leading into the first fully-connected layer.

VGG16 was made to solve ImageNet, and achieves a [8.8% top-5 error rate](https://github.com/jcjohnson/cnn-benchmarks), which means that 91.2% of test samples were classified correctly within the top 5 predictions for each image. It's top-1 accuracy--equivalent to the accuracy metric we've been using (that the top prediction is correct)--is 73%. This is especially impressive since there are not just 97, but 1000 classes, meaning that random guesses would get us only 0.1% accuracy.

In order to use this network for our task, we "remove" the final classification layer, the 1000-neuron softmax layer at the end, which corresponds to ImageNet, and instead replace it with a new softmax layer for our dataset, which contains 97 neurons in the case of the 101_ObjectCategories dataset.

In terms of implementation, it's easier to simply create a copy of VGG from its input layer until the second to last layer, and then work with that, rather than modifying the VGG object directly. So technically we never "remove" anything, we just circumvent/ignore it. This can be done in the following way, by using the keras `Model` class to initialize a new model whose input layer is the same as VGG but whose output layer is our new softmax layer, called `new_classification_layer`. Note: although it appears we are duplicating this large network, internally Keras is actually just copying all the layers by reference, and thus we don't need to worry about overloading the memory.

In [ ]:
# make a reference to VGG's input layer
inp = vgg.input

# make a new softmax layer with num_classes neurons
set_activation_5_new = 'softmax' # @param ["softmax", "relu", "sigmoid", "tanh"]
new_classification_layer = Dense(num_classes, activation = set_activation_5_new)

# connect our new layer to the second to last layer in VGG, and make a reference to it
out = new_classification_layer(vgg.layers[-2].output)

# create a new network between inp and out
model_new = Model(inp, out)


We are going to retrain this network, `model_new` on the new dataset and labels. But first, we need to freeze the weights and biases in all the layers in the network, except our new one at the end, with the expectation that the features that were learned in VGG should still be fairly relevant to the new image classification task. Not optimal, but most likely better than what we can train to in our limited dataset.

By setting the `trainable` flag in each layer false (except our new classification layer), we ensure all the weights and biases in those layers remain fixed, and we simply train the weights in the one layer at the end. In some cases, it is desirable to *not* freeze all the pre-classification layers. If your dataset has enough samples, and doesn't resemble ImageNet very much, it might be advantageous to fine-tune some of the VGG layers along with the new classifier, or possibly even all of them. To do this, you can change the below code to make more of the layers trainable.

In the case of CalTech-101, we will just do feature extraction, fearing that fine-tuning too much with this dataset may overfit. But maybe we are wrong? A good exercise would be to try out both, and compare the results.

So we go ahead and freeze the layers, and compile the new model with exactly the same optimizer and loss function as in our first network, for the sake of a fair comparison. We then run `summary` again to look at the network's architecture.

In [ ]:
# make all layers untrainable by freezing weights (except for last layer)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

In [ ]:
# Set Parameters
# Default: optmizer = "adam", loss = "categorical_crossentropy"

set_optmizer_new    = 'sgd' # @param ["adam", "rmsprop", "sgd"]
set_loss_new        = 'categorical_crossentropy' # @param ["categorical_crossentropy", "binary_crossentropy", "mse"]

model_new.compile(loss  = set_loss_new,
              optimizer = set_optmizer_new,
              metrics=['accuracy'])

model_new.summary()

Looking at the summary, we see the network is identical to the VGG model we instantiated earlier, except the last layer, formerly a 1000-neuron softmax, has been replaced by a new 97-neuron softmax. Additionally, we still have roughly 134 million weights, but now the vast majority of them are "non-trainable params" because we froze the layers they are contained in. We now only have 397,000 trainable parameters, which is actually only a quarter of the number of parameters needed to train the first model.

As before, we go ahead and train the new model, using the same hyperparameters (batch size and number of epochs) as before, along with the same optimization algorithm. We also keep track of its history as we go.

In [ ]:
# Set Parameters
# Default: batch_size = 128, epochs = 10
set_batch_size_new  = 128  # @param {type:"integer"}
set_epochs_new      = 50   # @param {type:"integer"}

history2 = model_new.fit(x_train, y_train,
                         batch_size = set_batch_size_new,
                         epochs     = set_epochs_new,
                         validation_data=(x_val, y_val))


Our validation accuracy hovers close to 80% towards the end, which is more than 30% improvement on the original network trained from scratch (meaning that we make the wrong prediction on 20% of samples, rather than 50%).

It's worth noting also that this network actually trains _slightly faster_ than the original network, despite having more than 100 times as many parameters! This is because freezing the weights negates the need to backpropagate through all those layers, saving us on runtime.

Let's plot the validation loss and accuracy again, this time comparing the original model trained from scratch (in blue) and the new transfer-learned model in green.

In [ ]:
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.plot(history2.history["val_loss"])
ax.set_title("validation loss")
ax.set_xlabel("epochs")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_accuracy"])
ax2.plot(history2.history["val_accuracy"])
ax2.set_title("validation accuracy")
ax2.set_xlabel("epochs")
ax2.set_ylim(0, 1)

plt.show()

Notice that whereas the original model began overfitting around epoch 16, the new model continued to slowly decrease its loss over time, and likely would have improved its accuracy slightly with more iterations. The new model made it to roughly 80% top-1 accuracy (in the validation set) and continued to improve slowly through 100 epochs.

It's possibly we could have improved the original model with better regularization or more dropout, but we surely would not have made up the >30% improvement in accuracy.

Again, we do a final validation on the test set.

In [ ]:
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

# PREDICTION
To predict a new image, simply run the following code to get the probabilities for each class.

In [ ]:
from google.colab import files

def predictModel(path):
  img, x = get_image(path)
  probabilities = model_new.predict([x], verbose = 1)

  # Print probabilities with class names
  for i, prob in enumerate(probabilities[0]):
      print(f"Probability for {categories[i].split('/')[-1]}: {prob:.4f}")
  return

### Upload File

In [ ]:
uploaded = files.upload(target_dir = "downloads")
path = list(uploaded.keys())[0]

In [ ]:
predictModel(path)

### Load from Drive

In [ ]:
path = "downloads/sample02.jpg"
predictModel(path)

### Improving the results

Achieving around 85% top-1 accuracy on a 2-class dataset is a good outcome, especially considering the relatively small size of the dataset. While this is much better than what could be achieved with a model trained from scratch on this limited data, there is still potential for further improvement. Here are some techniques that could potentially enhance the performance:

- **Using data augmentation:** Augmentation refers to applying various transformations to the original training data, such as rotations, scaling, cropping, and changes in brightness or contrast. This increases the effective size of the training set and helps the model generalize better to unseen images.
- **Using a different optimizer, adding more regularization/dropout, and other hyperparameters:** Experimenting with different optimization algorithms, adjusting dropout rates, and tuning other hyperparameters like learning rate can significantly impact model performance.
- **Training for longer:** In some cases, training for more epochs might lead to further improvements, provided the model is not overfitting.

A more advanced example of transfer learning in Keras, involving augmentation for a small 2-class dataset, can be found in the [Keras blog](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html).